In [ ]:

%pip install nltk scikit-learn numpy python-Levenshtein -q

import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)



True

In [ ]:
%pip install rank-bm25 -q


In [3]:
import string
import numpy as np
from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from Levenshtein import distance as levenshtein_distance

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans


In [5]:
from datasets import load_dataset, concatenate_datasets

full_dataset_dict = load_dataset("taln-ls2n/inspec")

dataset_to_eval = concatenate_datasets([
    full_dataset_dict['train'], 
    full_dataset_dict['validation'], 
    full_dataset_dict['test']
])

print(f"Jumlah total dokumen: {len(dataset_to_eval)}")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Jumlah total dokumen: 2000


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [6]:
dataset_to_eval

Dataset({
    features: ['id', 'title', 'abstract', 'keyphrases', 'prmu'],
    num_rows: 2000
})

In [7]:
import numpy as np
from collections import defaultdict
from datasets import load_dataset, concatenate_datasets
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from rank_bm25 import BM25Okapi
from Levenshtein import distance as levenshtein_distance

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

class BM25_RRF_SearchEngine:
    def __init__(self, dataset):
        self.dataset = list(dataset)
        self.doc_map = {doc['id']: doc for doc in self.dataset}

        tokenized_titles = [preprocess(doc.get('title', '')) for doc in self.dataset]
        tokenized_keyphrases = [preprocess(' '.join(doc.get('keyphrases', []))) for doc in self.dataset]
        tokenized_abstracts = [preprocess(doc.get('abstract', '')) for doc in self.dataset]
        
        self.vocabulary = set()
        for doc_tokens in tokenized_titles:
            self.vocabulary.update(doc_tokens)
        if any(tokenized_keyphrases):
            for doc_tokens in tokenized_keyphrases:
                self.vocabulary.update(doc_tokens)
        for doc_tokens in tokenized_abstracts:
            self.vocabulary.update(doc_tokens)
        print(f"   - Vocabulary dibuat dengan {len(self.vocabulary)} kata unik.")
        
        if any(tokenized_titles): self.bm25_title = BM25Okapi(tokenized_titles)
        else: self.bm25_title = None
            
        if any(tokenized_keyphrases): self.bm25_keyphrases = BM25Okapi(tokenized_keyphrases)
        else: self.bm25_keyphrases = None
            
        if any(tokenized_abstracts): self.bm25_abstract = BM25Okapi(tokenized_abstracts)
        else: self.bm25_abstract = None
            

    def _correct_query_word(self, word):
        if word in self.vocabulary:
            return word
        min_dist = float('inf')
        corrected_word = word
        for vocab_word in self.vocabulary:
            dist = levenshtein_distance(word, vocab_word)
            if dist < min_dist:
                min_dist = dist
                corrected_word = vocab_word
        return corrected_word if min_dist <= 3 else word

    def search(self, query, top_n=10, k=60):
        print(f"\n🔎 Mencari untuk query: '{query}'")
        query_tokens = preprocess(query)
        
        # --- PERUBAHAN LOGIKA PENCETAKAN TYPO DIMULAI DI SINI ---
        corrected_query_tokens = []
        corrections_made = [] # Untuk menyimpan detail koreksi
        typo_found = False # Penanda apakah ada typo

        for token in query_tokens:
            corrected_word = self._correct_query_word(token)
            if corrected_word != token:
                typo_found = True
                corrections_made.append(f"'{token}' -> '{corrected_word}'")
            corrected_query_tokens.append(corrected_word)
        
        # Setelah loop selesai, cetak satu pesan ringkasan JIKA ada typo
        if typo_found:
            print(f"    Mendeteksi dan mengoreksi typo: {', '.join(corrections_made)}")
        # --- PERUBAHAN LOGIKA SELESAI DI SINI ---

        title_rank_map, keyphrase_rank_map, abstract_rank_map = {}, {}, {}
        all_doc_indices = set()
        
        if self.bm25_title is not None:
            title_scores = self.bm25_title.get_scores(corrected_query_tokens)
            title_ranks_indices = np.argsort(title_scores)[::-1]
            title_rank_map = {doc_idx: rank + 1 for rank, doc_idx in enumerate(title_ranks_indices)}
            all_doc_indices.update(title_ranks_indices)

        if self.bm25_keyphrases is not None:
            keyphrase_scores = self.bm25_keyphrases.get_scores(corrected_query_tokens)
            keyphrase_ranks_indices = np.argsort(keyphrase_scores)[::-1]
            keyphrase_rank_map = {doc_idx: rank + 1 for rank, doc_idx in enumerate(keyphrase_ranks_indices)}
            all_doc_indices.update(keyphrase_ranks_indices)
            
        if self.bm25_abstract is not None:
            abstract_scores = self.bm25_abstract.get_scores(corrected_query_tokens)
            abstract_ranks_indices = np.argsort(abstract_scores)[::-1]
            abstract_rank_map = {doc_idx: rank + 1 for rank, doc_idx in enumerate(abstract_ranks_indices)}
            all_doc_indices.update(abstract_ranks_indices)
        
        rrf_scores = defaultdict(float)
        for doc_idx in all_doc_indices:
            score = 0.0
            if doc_idx in title_rank_map:
                score += 1 / (k + title_rank_map[doc_idx])
            if doc_idx in keyphrase_rank_map:
                score += 1 / (k + keyphrase_rank_map[doc_idx])
            if doc_idx in abstract_rank_map:
                score += 1 / (k + abstract_rank_map[doc_idx])
            rrf_scores[doc_idx] = score
            
        sorted_docs = sorted(rrf_scores.items(), key=lambda item: item[1], reverse=True)
        
        results = []
        for doc_idx, score in sorted_docs[:top_n]:
            original_doc = self.dataset[doc_idx]
            results.append({
                'id': original_doc['id'],
                'title': original_doc.get('title', 'N/A'),
                'score': score,
                'abstract': original_doc.get('abstract', 'N/A'),
                'keyphrases':  original_doc.get('keyphrases', 'N/A')
            })
            
        return results

In [8]:
engine_rrf = BM25_RRF_SearchEngine(dataset_to_eval)

query = "computer Visiom"
search_results = engine_rrf.search(query, top_n=5, k=60)

print("\n--- Hasil Pencarian ---")
for doc in search_results:
    print(f"\n  ID: {doc['id']}, Skor RRF: {doc['score']:.6f}")
    print(f"  Title: {doc['title']}")
    print(f"  Abstract: {doc['abstract'][:150]}...")
    print(f"  keyphrases:{doc['keyphrases']} ")

   - Vocabulary dibuat dengan 8909 kata unik.

🔎 Mencari untuk query: 'computer Visiom'
    Mendeteksi dan mengoreksi typo: 'visiom' -> 'vision'

--- Hasil Pencarian ---

  ID: 1103, Skor RRF: 0.041959
  Title: New age computing [autonomic computing]
  Abstract: Autonomic computing (AC), sometimes called self-managed computing, is the name chosen by IBM to describe the company's new initiative aimed at making ...
  keyphrases:['new age computing', 'autonomic computing', 'AC', 'self-managed computing', 'IBM initiative', 'computing reliability', 'problem-free computing', 'computer speed', 'computer memory', 'computer crash', 'IT industry initiatives', 'AC requirements', 'AC development', 'AC implementation', 'open standards', 'self-healing computing', 'adaptive algorithms'] 

  ID: 1785, Skor RRF: 0.041323
  Title: The effect of a male-oriented computer gaming culture on careers in the computer industry
  Abstract: If careers in the computer industry were viewed, it would be evident that